# Springboard Capstone 2: Tweet Sentiment Extraction

## Solution Approach 2: Deep Learning model treating this as QA problem

In [18]:
# imports
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import json

import re
import nltk

In [2]:
# Read data into memory
training_fn = r'../data/raw/train.csv'
test_fn = r'../data/raw/test.csv'

In [3]:
train_df = pd.read_csv(training_fn, index_col = 'textID')
test_df = pd.read_csv(test_fn, index_col = 'textID')

In [4]:
def drop_missing_rows(df_in):
    df_out = df_in.dropna(axis = 0)
    return df_out  

In [5]:
def text_preprocess(text):
    # Strip leading and lagging whitespace
    text = text.strip()
    
    # Remove accented characters
#     text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Convert all text to lower case
    text.lower()
    
    # Take raw text and remove all audience reactions
#     text = re.sub('\((.*?)\)', '', text)

    # Remove url's from the tweets.
#     text = re.sub(r'(https?://\S+)', '', text)
    return text

In [6]:
# Drop missing data
train_df = drop_missing_rows(train_df)

Need to add a prepocessing step where selected text is NaN for input text of NaN

In [7]:
train_df['clean_text'] = train_df.text.apply(text_preprocess)

In [8]:
train_df.head(5)

,text,selected_text,sentiment,clean_text
textID,,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going"
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!
088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...
9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th..."


# DistilBERT Implementation

https://www.kaggle.com/doomdiskday/full-tutorial-eda-to-dnns-all-you-need/comments#Baseline-Models

Uses model : distilbert-base-uncased-distilled-squad model

We use pre-trained model fine-tuned on SQuAD, Stanford Question Answering Dataset

In [ ]:
!mkdir -p data

In [22]:

from simpletransformers.question_answering import QuestionAnsweringModel
from copy import deepcopy
use_cuda = False

In [12]:
!pip install '/kaggle/input/simple-transformers-pypi/seqeval-0.0.12-py3-none-any.whl' -q
!pip install '/kaggle/input/simple-transformers-pypi/simpletransformers-0.22.1-py3-none-any.whl' -q

ERROR: Invalid requirement: "'/kaggle/input/simple-transformers-pypi/seqeval-0.0.12-py3-none-any.whl'"
ERROR: Invalid requirement: "'/kaggle/input/simple-transformers-pypi/simpletransformers-0.22.1-py3-none-any.whl'"


In [16]:
def read_train():
    train=pd.read_csv("../data/raw/train.csv")
    train['text']=train['text'].astype(str)
    train['selected_text']=train['selected_text'].astype(str)
    return train

def read_test():
    test=pd.read_csv("../data/raw/test.csv")
    test['text']=test['text'].astype(str)
    return test

def read_submission():
    test=pd.read_csv("../data/raw/sample_submission.csv")
    return test

In [23]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

NameError: name 'logging' is not defined

In [24]:
def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

In [25]:
def do_qa_train(train):

    output = {}
    output['version'] = 'v1.0'
    output['data'] = []
    paragraphs = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]
        if type(answer) != str or type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break

        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        paragraphs.append({'context': context.lower(), 'qas': qas})
        output['data'].append({'title': 'None', 'paragraphs': paragraphs})
        
    return paragraphs

In [26]:
def do_qa_test(test):
    paragraphs = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        paragraphs.append({'context': context.lower(), 'qas': qas})
        output['data'].append({'title': 'None', 'paragraphs': paragraphs})
    return paragraphs

In [38]:
def post_process(submission_df, test_df):

    index_to_selected_text = {}
    for i, row in test_df.iterrows():
        _id = row[0]
        text = row[1]
        sentiment = row[2]
        if len(text.split(" ")) <= 3 or sentiment == "neutral":
            index_to_selected_text[_id] = text
    
    submission_rows = submission_df.to_dict("records")
    new_rows = []
    for row in submission_rows:
        _id = row['textID']
        if _id in index_to_selected_text:
            new_row = deepcopy(row)
            new_row['selected_text'] = index_to_selected_text[_id]
        else:
            new_row = row
        
        new_rows.append(new_row)

    return pd.DataFrame(new_rows)

In [27]:

train_df = read_train()
test_df = read_test()
submission_df_distil = read_submission()

In [28]:
train = np.array(train_df)
test = np.array(test_df)

In [29]:
qa_train = do_qa_train(train)

In [31]:
with open('data/train.json', 'w') as outfile:
    json.dump(qa_train, outfile)

In [32]:
output = {}
output['version'] = 'v1.0'
output['data'] = []

In [34]:
qa_test = do_qa_test(test)
with open('data/test.json', 'w') as outfile:
    json.dump(qa_test, outfile)

In [35]:
model = QuestionAnsweringModel('distilbert', 
                               'distilbert-base-uncased-distilled-squad', 
                              args={"reprocess_input_data": True,
                               "overwrite_output_dir": True,
                               "learning_rate": 8e-05,
                               "num_train_epochs": 3,
                               "max_seq_length": 192,
                               "weight_decay": 0.001,
                               "doc_stride": 64,
                               "save_model_every_epoch": False,
                               "fp16": False,
                               "do_lower_case": True,
                                 'max_query_length': 8,
                               'max_answer_length': 150
                                    },
                              use_cuda=use_cuda)

In [36]:
model.train_model('data/train.json')
predictions = model.predict(qa_test)
predictions_df = pd.DataFrame.from_dict(predictions)

submission_df_distil['selected_text'] = predictions_df['answer']


100%|██████████████████████████████████████████████████████████████████████████| 27481/27481 [00:25<00:00, 1061.51it/s]


Running loss: 1.366487

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.920035


Running loss: 0.002089


Running loss: 0.852820



100%|████████████████████████████████████████████████████████████████████████████| 3534/3534 [00:03<00:00, 1117.95it/s]


NameError: name 'post_process' is not defined

In [37]:
submission_df_distil.selected_text

0                                 last session of the day
1                                                exciting
2                                                  shame!
3                                             happy bday!
4                                                  i like
                              ...                        
3529                                                tired
3530    thanks for the net which keeps me alive and ki...
3531         . my little dog is sinking into depression..
3532                                               i love
3533                                      omgssh ang cute
Name: selected_text, Length: 3534, dtype: object

In [39]:
submission_df_distil = post_process(submission_df_distil, test_df)

In [40]:
submission_df_distil

,textID,selected_text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,exciting
2,eee518ae67,shame!
3,01082688c6,happy bday!
4,33987a8ee5,i like
...,...,...
3529,e5f0e6ef4b,tired
3530,416863ce47,thanks for the net which keeps me alive and ki...
3531,6332da480c,. my little dog is sinking into depression..
3532,df1baec676,i love


Lets also get training predictions

In [41]:
predictions_train = model.predict(qa_train)
predictions_df_train = pd.DataFrame.from_dict(predictions_train)



100%|██████████████████████████████████████████████████████████████████████████| 27481/27481 [00:22<00:00, 1236.36it/s]


In [42]:
predictions_df_train

,id,answer
0,cb774db0d1,"i`d have responded, if i were going"
1,549e992a42,sad
2,088c60f138,bullying
3,9642c003ef,leave me alone
4,358bd9e861,"sons of ****,"
...,...,...
27476,4eac33d1c0,husband lost
27477,4f4c4fc327,", don`t force"
27478,f67aae2310,yay good for both of you.
27479,ed167662a5,worth


In [43]:
train_df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [47]:
predictions_train = train_df.merge(predictions_df_train,left_on = 'textID', right_on = 'id' )

In [48]:
predictions_train.head()

,textID,text,selected_text,sentiment,id,answer
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,cb774db0d1,"i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,549e992a42,sad
2,088c60f138,my boss is bullying me...,bullying me,negative,088c60f138,bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,9642c003ef,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,358bd9e861,"sons of ****,"


In [46]:
def jaccard(str1, str2): 
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    jaccard_score = float(len(c)) / (len(a) + len(b) - len(c))
    return jaccard_score
vec_jaccard = np.vectorize(jaccard)

In [56]:
jaccard_trivial = vec_jaccard(predictions_train.text, predictions_train.selected_text)
jaccard_distilbert = vec_jaccard(predictions_train.selected_text, predictions_train.answer)

In [50]:
print ('Trivial jaccard score(between text and selected text is',np.mean(jaccard_trivial) )

Trivial jaccard score(between text and selected text is 0.5890913411191024


In [57]:
print ('Predicted jaccard score(between selected text and predicted answer is',np.mean(jaccard_distilbert) )

Predicted jaccard score(between selected text and predicted answer is 0.8860384090658026


Lets write to file for postprocessing as needed

In [77]:
predictions_train.to_csv(r'../data/processed/distilbert/predictions_distilbert.csv')

In [76]:
submission_df_distil.to_csv(r'../data/processed/distilbert/training_submission.csv')

In [63]:
submission_df_distil.head()

,textID,selected_text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,exciting
2,eee518ae67,shame!
3,01082688c6,happy bday!
4,33987a8ee5,i like


In [62]:
test_df.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [71]:
test_predictions = test_df.merge(submission_df_distil,on = 'textID')

In [75]:
test_predictions.rename(columns = {'selected_text':'selected_text_ans'}, inplace = True)
test_predictions.to_csv(r'../data/processed/distilbert/test_predictions.csv')